In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import time
import json
import pandas as pd


In [ ]:
super_response_list = []
misconfigured_props = []
errors = []
output_file_name = "99acres_data.xlsx"

In [ ]:
amenities_map = {
	'1': 'Swimming Pool',
	'2': 'Power Backup',
	'3': 'Club house',
	'4': 'Parking',	
	'5' : 'Feng Shui / Vaastu Compliant',
	'6': 'Park',	
  '8' : 'Private Garden / Terrace',
	'9': 'Security Personnel',	
  '10' : 'Centrally Air Conditioned',
	'11': 'ATM',
	'12': 'Fitness Centre / GYM',
	'17' : 'Security/Fire Alarm',
	'19': 'Visitor Parking',
	'20' : 'Intercom Facility',
	'21': 'Lift',
	'23' : 'Maintenance Staff',
	'24' : 'Water Storage',	
	'25': 'Waste disposal',
	'26' : 'Rain Water Harvesting',
	'28': 'Bank Attached Property',
	'29': 'Piped-gas',
	'30': 'Internet/wi-fi connectivity',
	'31': 'Water softening plant',
  '32': 'Water purifier',
	'33': 'Shopping Centre',
	'34': 'Wheelchair Accessibility',
	'35': 'DG Availability',
	'39' : 'Recently Renovated',
	'40' : 'Natural Light',
	'41' : 'Airy Rooms',
  '42' : 'Spacious Interiors',
	'43' : 'Low Density Society',
	'44' : 'High Ceiling Height',
	'45' : 'False Ceiling Lighting',
  '46' : 'Separate entry for servant room',
	'47' : 'No open drainage around',
  '101' : 'Wheelchair Friendly',
  '102' : 'Pet Friendly',
  '103' : 'AC Room',
  '104' : 'Food Service',
  '105' : "Wifi",
  '106' : 'Laundry Available'
}

In [1]:
url_template = 'https://www.99acres.com/api-aggregator/discovery/srp/search?sortby=date_d&area_unit=1&platform=DESKTOP&moduleName=GRAILS_SRP&workflow=GRAILS_SRP&page_size=200&page={}&city=38&preference=S&res_com=R&seoUrlType=DEFAULT'
resp_list = []
for i in range(1,100) :
  url = url_template.format(i)
  
  try : 
    resp = requests.get(url)
    time.sleep(0.5)
    print(i,resp)
    resp_json = resp.json()
    resp_list.append(resp_json)
    continue
  except Exception as e:
    print(i,resp,e)
    continue
len(resp_list)
super_response_list.extend(resp_list)
with open(f'99acres_{i}_responses_data.json', 'w') as fp:
    saved_json = {
        'responses' : super_response_list
    }
    json.dump(saved_json, fp)



NameError: ignored

In [ ]:
len(resp_list)

99

In [ ]:
final_results = []
c = 0
for resp in resp_list:
  results,c = get_properties_of_a_resp(resp,c)
  final_results.extend(results)
  # print("len",len(results))

print("Final length",len(final_results))
# print("ACtual prop count",c)

final_df = pd.DataFrame(final_results)
final_df.to_excel(output_file_name)

uniq_err = set(errors)
print(len(uniq_err))


In [ ]:
def get_properties_of_a_resp(response_json, c) :
  page_json_list = []
  # response = requests.get(url2)
  # print(response,response.text)
  # response_json = response.json()
  # response_json.keys()
  facets = response_json['facets']
  properties = response_json['properties']
  amenities_list = response_json['facets']['FEATURES']
  availability_list = response_json['facets']['SUB_AVAILABILITY']


# fills the page_json
  for prop in properties :
    c = c+1
    prop_dets = get_property_details(prop)
    if prop_dets == None :
      continue
    page_json_list.append(prop_dets)
    
  print("Number of good props in this json : ",len(page_json_list))

  return page_json_list,c

In [ ]:
def get_property_details(prop) :
  url_prefix = 'https://www.99acres.com'
  details = {}

  try :
    details['Apartment Name'] = prop.get('BUILDING_NAME','Null')
    details['Society Name'] = prop['location'].get('SOCIETY_NAME','Null')
    details['Location'] = prop['location']['LOCALITY_NAME']

    details['Heading'] = prop['PROP_HEADING']
    details['URL'] = url_prefix + prop.get('PD_URL','Null')
    details['Property ID'] = prop['PROP_ID']
    details['Property Type'] = prop['PROPERTY_TYPE']
    details['Photo Url'] = prop.get('MEDIUM_PHOTO_URL','Null')
    


    details['Registered Date'] = prop['REGISTER_DATE__U']
    

    address = prop['location'].get('BUILDING_NAME','Null') + "," + prop['location'].get('LOCALITY_NAME','Null') + "," + prop['location'].get('CITY_NAME','Null')
    details['Address'] = address
    details['Area'] = prop['AREA']
    avail_id = prop['FORMATTED']['AVAIL']['AVAILABILITY']
    details['Availability'] = 'Ready to move' if avail_id == 2 else 'Under Construction'
    details['Price in Rs'] = prop['FORMATTED']['AVG_PRICE']
    details['Price in words'] = prop['FORMATTED']['PRICE_IN_WORDS']
    details['Owner/Dealer Name'] = prop.get('CONTACT_NAME','Null')

    details['Price Per unit area'] = prop.get('PRICE_PER_UNIT_AREA','Null')

    facing = {
        1 : 'NORTH',
        2 : 'SOUTH',
        3 : 'EAST',
        4 : 'WEST',
        5 : 'North-East',
        'Null' : 'Null'
    }    

    details['Facing'] = facing.get(prop.get('FACING','Null'),'Unknown')
    
  

    details['Total Floors'] = prop.get('TOTAL_FLOOR','Null')
    details['Floor Number'] = prop.get('FLOOR_NUM','Null')
    details['Bedroom'] = prop.get('BEDROOM_NUM','Null')
    details['Bathroom'] = prop.get('BATHROOM_NUM','Null')
    details['Balcony'] = prop.get('BALCONY_NUM','Null')
    details['Rera Status'] = prop['xid'].get('REGISTRATION_STATUS','Null')
    details['Rera Number'] = prop['xid'].get('PROJ_RERA_REGISTRATION_ID','Null')


    am_ids = prop['AMENITIES'].split(',')
    ams_text = [amenities_map.get(id,id) for id in am_ids]
    details['Amenities'] = ams_text
    

    return details
  
  except Exception as e:
    misconfigured_props.append(prop)
    errors.append(str(e))
    return None
  

In [ ]:
ids = [47,28,29,30,39,19,1,6,9,40,41,43,25,26,31,33,12,3]
ams_text = '''
No open drainage around
Bank Attached Property
Piped-gas
Internet/wi-fi connectivity
Recently Renovated
Visitor Parking
Swimming Pool
Park
Security Personnel
Natural Light
Airy Rooms
Low Density Society
Waste Disposal
Rain Water Harvesting
Water softening plant
Shopping Centre
Fitness Centre / GYM
Club house / Community Center
'''
ams = ams_text.split("\n")[1:-1]
len(ams)

map = {}

for i in range(len(ams)):
  map[ids[i]] = ams[i] 

map


dict

In [ ]:
# response = requests.get("http://api.open-notify.org/astros.json")
url2 = 'https://www.99acres.com/api-aggregator/discovery/srp/search?sortby=date_d&area_unit=1&platform=DESKTOP&moduleName=GRAILS_SRP&workflow=GRAILS_SRP&page_size=25&page=750&city=38&preference=S&res_com=R&seoUrlType=DEFAULT'
response = requests.get(url2)
print(response)
print(response.headers)
print(response.text)
